### RNN e Word Embedding con Keras
Le reti neurali ricorrenti (RNNs - Recurrent Neural Networks) sono un'architettura di reti neurali artificiali che mantenedo una memoria interna ci permettono di analizzare sequenze temporale di dati.
<br><br>
In questo costruiremo una rete neurale ricorrente per un problema di sentiment analysis, utilizzando sempre il dataset l'Internet Movie Database.
<br><br>
Carichiamo i moduli che ci serviranno.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


## Scarichiamo il dataset
Utilizziamo Keras per caricare l'imdb dataset, limitandolo alle 10000 parole più comuni.

In [2]:
from keras.datasets import imdb 

num_words = 10000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))

17465344/17464789 [==============================] - 5s 0us/step
Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


## Preprocessiamo i dati
Per rappresentare le nostre sequenze utilizzeremo il Word Embedding, che va aggiunto come strato iniziale della nostra rete neurale.
Le recensioni all'interno del corpus di testo hanno ovviamente lunghezza differente


In [3]:
longest_review = max(X_train,key=len)
shortest_review = min(X_train,key=len)

print("La review più lunga ha %d parole" % len(longest_review))
print("La review più corta ha %d parole" % len(shortest_review))

La review più lunga ha 2494 parole
La review più corta ha 11 parole


trochiamole dopo 100 parole utilizzando la funzione pad_sequence di keras. Se una recensione ha meno di 100 parole verranno aggiunti una serie di zeri al termine per portarla alla lunghezza corretta.

In [4]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_train = pad_sequences(X_train, maxlen = maxlen)
X_test = pad_sequences(X_test, maxlen = maxlen)

X_train.shape

(25000, 100)

## Creiamo il modello
Il primo strato del nostro modello eseguirà il Word Embedding. Il Word embedding è un modello che ci permette di generare una serie di vettori (embedding vectors) ognuno dei quali quantifica una caratteristica delle parole. Questo tipo di rappresentazione che tiene conto di relazioni e dipendenze tra le parole è un ottimo input per una RNN.
<br><br>
La classe Embedding di Keras ha due parametri:
 - 1: Il numero di parole nel nostro dizionario
 - 2: Il numero di embedding vectors da generare
 
 
Per aggiungere lo strato ricorrente alla nostra rete possiamo usare la classe SimpleRNN.
 

In [5]:
from keras.layers import LSTM, SimpleRNN, Embedding

model = Sequential()

model.add(Embedding(num_words, 32))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


Compiliamo il modello ed eseguiamo l'addestramento per sole 10 epoche.
<br>
Come algoritmo di ottimizzazione per reti neurali ricorrenti Keras consiglia di utilizzare **RMSprop**, come puoi leggere [qui](https://keras.io/optimizers/). Non sono riuscito a trovare evidenze sperimentali per questa affermazione (e non sono l'unico, come puoi vedere [qui](https://github.com/keras-team/keras/pull/12469) e [qui](https://github.com/keras-team/keras/issues/12460)), ma seguiamo il consiglio di Keras e utilizziamo **RMSprop**. 

In [6]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=512, validation_split=0.2, epochs=10)
model.evaluate(X_test, y_test)

Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 5s 230us/step - loss: 0.6764 - acc: 0.5690 - val_loss: 0.6599 - val_acc: 0.5814
Epoch 2/10
20000/20000 [==============================] - 2s 98us/step - loss: 0.5247 - acc: 0.7719 - val_loss: 0.5257 - val_acc: 0.7504
Epoch 3/10
20000/20000 [==============================] - 2s 99us/step - loss: 0.3933 - acc: 0.8451 - val_loss: 0.4610 - val_acc: 0.7964
Epoch 4/10
20000/20000 [==============================] - 2s 103us/step - loss: 0.3080 - acc: 0.8845 - val_loss: 0.5117 - val_acc: 0.7444
Epoch 5/10
20000/20000 [==============================] - 2s 99us/step - loss: 0.2511 - acc: 0.9087 - val_loss: 0.5416 - val_acc: 0.7796
Epoch 6/10
20000/20000 [==============================] - 2s 100us/step - loss: 0.1967 - acc: 0.9331 - val_loss: 0.5125 - val_acc: 0.8020
Epoch 7/10
20000/20000 [==============================] - 2s 98us/step - loss:

[0.5250912714862823, 0.7886]

I risultati della nostra rette ricorrente sono scarsi se confrontati a quelli che abbiamo ottenuto in precedenza con un semplice percettrone multistrato. Inoltre  il modello soffre di un problema di un evidente problema di overfitting sul set di addestramento.
<br>
Le reti ricorrenti soffrono del problema della **scomparsa del gradiente** tra le diverse esecuzioni della rete e questo le rende inadatte per sequenze di dati troppo lunghe.
<br><br>
In questi casi la soluzione migliore è utilizzare le **Long short-term memory** network.